##### Install libraries:

In [3]:
!pip3 install pdfplumber
!pip3 install openai==1.55.3 -q

!pip3 install pdf2image
!apt-get install poppler-utils
!pip3 install easyocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.10).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


##### Import libraries:

In [4]:
from openai import OpenAI
from PIL import Image

import os, time, json
import numpy as np
import pandas as pd

from tqdm import tqdm

from google.colab import drive
from google.colab import output
from google.colab import files

from joblib import Parallel, delayed
from concurrent.futures import ThreadPoolExecutor, as_completed
from IPython.display import display

import nltk, re
from nltk.tokenize import sent_tokenize

nltk.download('punkt')
nltk.download('punkt_tab')

from tqdm import tqdm

from pdf2image import convert_from_path
import easyocr, pdfplumber
import warnings, logging

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


##### Ignore warnings:

In [5]:
warnings.filterwarnings("ignore")
logging.getLogger("pdfminer").setLevel(logging.ERROR)
Image.MAX_IMAGE_PIXELS = None

##### Keys:

In [7]:
os.environ["OPENAI_API_KEY"] = ""

##### UDFs:

In [13]:
def extract_field(text, prompt, counter=0, model="o3", system_prompt = None):

    client = OpenAI()

    system_prompt = "Ты эксперт по технологическим стартапам и смотришь презентацию стартапа для венчурного фонда"

    messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": f'task: {prompt} : "text: {text}" '}]
    response = client.chat.completions.create(model=model, messages=messages).choices[0].message.content

    return response

def return_fields(doc_txt):

    startup_fields = []

    z = extract_field(doc_txt, 'извлеки отсюда 1. Описание компании - если нет, выведи только одно слово НЕТ;\
     2. Финансовые показатели компании - если нет, выведи только одно слово НЕТ;\
     3. Информацию о команде компании (опыт в разработке и бизнесе) - если нет, выведи только одно слово НЕТ;\
     4. Привлеченные государственные гранты и финансирование - если нет, выведи только одно слово НЕТ;\
     5. Описание продукта и его стадии готовности (например есть прототип, есть продажи) - если нет, выведи только одно слово НЕТ;\
     6. Описание используемых технологий - если нет, выведи только одно слово НЕТ;\
     7. Объем рынка продукта - если нет, выведи только одно слово НЕТ;\
     8. Технологические тренды рынка - если нет, выведи только одно слово НЕТ;\
     9. Риски рынка, для которого предназначен продукт - если нет, выведи только одно слово НЕТ;\
     10. Технологические риски продукта - если нет, выведи только одно слово НЕТ;\
     11. Аналоги продукта в мире - если нет, выведи только одно слово НЕТ;\
     12. Конкуренты продукта - если нет, выведи только одно слово НЕТ;\
     13. Сделки в мире по таким типам продукта - если нет, выведи только одно слово НЕТ;\
     14. Сделки в России по таким типам продукта - если нет, выведи только одно слово НЕТ;\
     15. Привлеченные инвестиции - если нет, выведи только одно слово НЕТ;\
     инвестиционное предложение - если нет, выведи только одно слово НЕТ; между ними разделение символом #')

    return [re.sub(r"^\d+\.\s*", "", re.sub(r"^\d+\)\s*", "", i.strip().replace('\n', ''))) for i in z.split('#')]

def parallel_extraction(texts, max_workers: int = 10):
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        results = list(tqdm(executor.map(lambda x: return_fields(x), texts), total=len(texts)))
    return results

##### Extract files:

In [11]:
drive.mount('/content/drive')
# !cp -r '/content/drive/My Drive/Центр Аналитики/IT системы/ML модели/Саммаризатор/Питчи' '/content/'
!cp -r '/content/drive/My Drive/Pitchdecks' '/content/'

Mounted at /content/drive


##### Extract fields:

In [14]:
%%time

folder_path = '/content/Pitchdecks'
pdf_files = sorted([f for f in os.listdir(folder_path) if f.endswith('.pdf')])

print('\nPDF text recognition...\n')

df_fields, file_texts = [], []
adresses = []

for pdf_file in tqdm(pdf_files):

    pdf_path = os.path.join(folder_path, pdf_file)

    ### Image recognition ###
    pdf_images = convert_from_path(pdf_path, thread_count=10)

    for idx, image in enumerate(pdf_images):
      image.save(f'pdf_page_{idx + 1}.png', 'PNG')

    image_dir = '/content/'
    reader = easyocr.Reader(['ru', 'en'], gpu = True)

    text_results = []

    for filename in sorted(os.listdir(image_dir)):

        if filename.lower().endswith('.png'):

          try:
            img_path = os.path.join(image_dir, filename)
            result = reader.readtext(img_path, detail=0, paragraph=True)
            text_results.extend(result)
          except:
            print("Can't recognize the image")

    img_text = '\n'.join(text_results)
    !rm /content/*.png
    ###/Image recognition ###

    adresses.append(pdf_path.replace('/content/Питчи/', ''))

    # Extract text with PDF reader
    with pdfplumber.open(pdf_path) as pdf:
      text = ''
      for page in pdf.pages:
          text += page.extract_text() + '\n'

    doc_txt = text.replace('02.12.2024, 22:35 Документ от auto-coder.net', '')
    #/Extract text with PDF reader

    file_texts.append(img_text + '|Краткое содержание текста: ' + doc_txt)

print('\nFields extraction with O3...\n')

df_fields = list(parallel_extraction(file_texts))
df_fields = [[i for i in sublist if i != ''] for sublist in df_fields]
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')


PDF text recognition...



  0%|          | 0/1 [00:00<?, ?it/s]WARNING:easyocr.easyocr:Downloading detection model, please wait. This may take several minutes depending upon your network connection.


Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.1% Complete

100%|██████████| 1/1 [00:15<00:00, 15.95s/it]



Fields extraction with O3...



100%|██████████| 1/1 [00:13<00:00, 13.12s/it]


CPU times: user 12.2 s, sys: 1.27 s, total: 13.5 s
Wall time: 30.5 s


##### Show dataframe:

In [15]:
df = pd.DataFrame(df_fields, columns=['О компании', 'Показатели компании', 'Команда', 'Привлеченные меры поддержки', 'Продукт', 'Технология', 'Рынок', 'Технологические тренды рынка',\
                                      'Рыночные риски', 'Технологические риски', 'Аналоги в мире', 'Конкуренты', 'Сделки в Мире', 'Сделки в России', 'Привлеченные инвестиции', 'Инвестиционное предложение'])


for col in df.columns:
    if col != 'Файл':
        df[col] = df[col].str.capitalize()

df['Файл'] = adresses
print(df.to_markdown(index=False, tablefmt="grid"))

+-------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------+-----------+-------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------+--------------------------------+------------------+-------------------------+------------------+--------------+-----------------+-------------------+---------------------------+------------------------------+------------------------------+
| О компании                                                                             

##### Save to xlsx:

In [ ]:
df.to_excel('pitches.xlsx', index=False)
files.download('pitches.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>